In [ ]:
#include <stdlib.h>
#include <iostream>
#include <fstream>
#include <string>
using namespace std;
{
    system("~/bin/acta parsweep.hpp > out.txt");
    ifstream file("out.txt"); for (string line; getline(file, line); )cout << line << endl;
    system("rm out.txt");
}

In [ ]:
#pragma cling load("libcurl")
#include "../../lib/everest.hpp"
{
    templet::everest_engine teng("everest-login","everest-password");
    teng.save_access_token();
    if (teng) { string token; teng.get_access_token(token); std::cout << token;}
    else {std::cout << "--- authentication failed ---";}
}

In [ ]:
#pragma cling load("libcurl")
#include "../../lib/everest.hpp"
{
    templet::everest_engine teng("everest-access-token");
    teng.print_app_description("lyapunov");
}

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/")
#include "parsweep.hpp"
#include <chrono>
#include <random>
{
   
	app_ID = (char*)"everest-application-ID";
    maple_code = "./everest-app/code.txt";
	
	NUM_TASKS = 2;
	// set NUM_WORKERS in 'parsweep.hpp' if needed

	rho = new double[NUM_TASKS];
	sigma = new double[NUM_TASKS];
	beta = new double[NUM_TASKS];
    
    std::random_device rd;
    std::mt19937 gen(rd());
    
    std::uniform_real_distribution<> dist_rho(-1.0,1.0);
    std::uniform_real_distribution<> dist_sigma(-1.0,1.0);
    std::uniform_real_distribution<> dist_beta(-1.0,1.0);
    
    for(int i=0; i<NUM_TASKS; i++){
        rho[i]   = 28.0 + dist_rho(gen);
        sigma[i] = 10.0 + dist_sigma(gen);
        beta[i]  = 2.66666 + dist_beta(gen);
    }

	templet::engine eng;
	templet::everest_engine teng("everest-access-token");
 
	master mst(eng);
	worker wks[NUM_WORKERS];

	for (int i = 0; i < NUM_WORKERS; i++) {
		wks[i].engines(eng, teng);
		mst.r(wks[i].r);
	}
    
    auto start = std::chrono::high_resolution_clock::now();
	eng.start();
	teng.run();
    auto end = std::chrono::high_resolution_clock::now();
	
   if (!eng.stopped()) {
		templet::everest_error err;
		if (teng.error(&err)) templet::everest_engine::print_error(&err);
		else cout << "Logical error(((";
	}
    else{ 
        cout << "Success!!!\n";
        std::chrono::duration<double> diff = end-start;
        cout << "Computation time is " << diff.count() << " seconds";
    }
}